## Genetski algoritem

Z genetskim algoritmom želiva ovreči spodnjo neenakost

$ Ra(G) \geq rad(G) -1$

In [1]:
def randic(graf):
    '''vrne randicev indeks za podan graf'''
    vsota = 0
    povezave = graf.edges()
    stopnje = graf.degree()
    for edge in povezave:
        u, v,_ = edge
        d_u = stopnje[u]
        d_v = stopnje[v]
        vsota += 1/((d_u * d_v)**(1/2))
    return(vsota)

In [2]:
def fitness(graf):
    '''Vrne vrednost naše neenakosti, če je negativna lema ne drži'''
    return randic(graf) - graf.radius() + 1

In [14]:
def tournament_selection(populacija, t):
    '''med t naključno izbranimi grafi izbere tistega z najmanšim fitnesom'''
    velikost_populacije = len(populacija)
    n = randint(0, velikost_populacije - 1)
    najbolsi = populacija[n]
    fitnes_najbolsi = fitness(najbolsi)
    for i in range(1, t + 1):
        n = randint(0, velikost_populacije - 1)
        naslednji = populacija[n]
        fitnes = fitness(naslednji)
        if fitnes < fitnes_najbolsi:
            najbolsi = naslednji
            fitnes_najbolsi = fitnes
    return najbolsi

Z poissonom bomo izbirali število povezav, ki jih bomo v funkciji mutiraj odstranili oziroma dodali in v funkciji crossover nam pove koliko povezav bomo dodali potomcu

In [4]:
def poisson(t = 1, lambd = 1/2):
    '''poissonova porazdelitev'''
    N = 0
    S = 0
    while S < t:
        N += 1
        S += expovariate(lambd)
    return N

In [5]:
def mutiraj(graf):
    '''funkcija mutira graf, če p = 1/3 doda povezavo, če p = 1/3 odstrani povezavo in če p = 1/3 doda in odstrani povezavo'''
    kopija = Graph(graf)
    verjetnost = random()
    plus_povezave = poisson(lambd = 2) # max število povezav, ki jih bomo dodali
    minus_povezave = poisson(lambd = 2) # max število povezav, ki jih bomo odstranili
    if verjetnost <= 1/3:
        for k in range(plus_povezave):
            u, v = kopija.random_vertex(), kopija.random_vertex()
            if u != v:
                kopija.add_edge(u, v)
        return kopija
    elif verjetnost > 1/3 and verjetnost <= 2/3:
        for k in range(minus_povezave):
            povezava = kopija.random_edge()
            kopija.delete_edge(povezava)
            if not kopija.is_connected(): # če graf ni več povezan ko odstranimo povezavo jo moramo dodati nazaj
                kopija.add_edge(povezava)
        return kopija
    elif verjetnost > 2/3:
        for k in range(plus_povezave):
            u, v = kopija.random_vertex(), kopija.random_vertex()
            if u != v:
                kopija.add_edge(u, v)
        for k in range(minus_povezave):
            povezava = kopija.random_edge()
            kopija.delete_edge(povezava)
            if not kopija.is_connected():
                kopija.add_edge(povezava)
        return kopija

In [6]:
def crossover(a, b):
    '''križa dva grafa med sabo in vrne njunega potomca'''
    n = len(a)
    while True:
        podgraf_a = a.random_subgraph(0.5) # vrne podgraf grafa a, kjer je vsako vozlišče vsebovano z verjetnostjo 1/2
        podgraf_b = b.random_subgraph(0.5) # vrne podgraf grafa b, kjer je vsako vozlišče vsebovano z verjetnostjo 1/2
        if len(podgraf_a.vertices()) + len(podgraf_b.vertices()) == n and len(podgraf_a.vertices()) >= 1 and len(podgraf_a.vertices()) < n and podgraf_a.is_connected() and podgraf_b.is_connected():
            # z zgornjim pogojem želimo da sta oba podgraf povezana in da imata skupaj n vozlišč in da ni en podgraf prazen drugi pa ima n vozlišč
            podgraf_a.relabel()
            podgraf_b.relabel()
            potomec = podgraf_a.disjoint_union(podgraf_b) # povezave ki niso skupne med grafoma
            nove_povezave = poisson(lambd = log(n/2)) # večji kot je n več povezav bomo dodal
            for k in range(nove_povezave):
                u, v = podgraf_a.random_vertex(), podgraf_b.random_vertex()
                potomec.add_edge((0, u), (1, v))
            potomec.relabel()
            break
    return potomec

In [7]:
def zacetna_populacija(velikost, n):
    '''naredi začetno populacijo kjer imajo grafi n vozlišč'''
    populacija = []
    trenutna_velikost = 0
    while trenutna_velikost < velikost:
        graf = graphs.RandomGNP(n, random()) # naredi nek naključen graf z n vozlišči vsaka povezava je v grafu z neko naključno verjetnostjo
        if graf.is_connected():
            populacija.append(graf)
            trenutna_velikost += 1
    return populacija

In [8]:
def min_fitness(seznam):
    '''v seznamu grafov poišče graf z najmanšim fitnesom'''
    najbolsi = seznam[0]
    fitnes = fitness(najbolsi)
    for graf in seznam:
        fit = fitness(graf)
        if fit < fitnes:
            fitnes = fit
            najbolsi = graf
    return najbolsi

In [9]:
def nova_populacija(populacija,t):
    '''z križanjem in mutacijo grafov iz podane populacije naredi novo populacijo'''
    nova_populacija = []
    velikost_nove = 0
    velikost_stare = len(populacija)
    while velikost_nove < velikost_stare:
        graf1 = tournament_selection(populacija, t)
        graf2 = tournament_selection(populacija, t)
        mutacija1 = mutiraj(graf1)
        mutacija2 = mutiraj(graf2)
        krizan_graf = crossover(mutacija1, mutacija2)
        optimalen_graf = min_fitness([krizan_graf, mutacija1, mutacija2, graf1, graf2])
        nova_populacija.append(optimalen_graf)
        velikost_nove += 1
    return nova_populacija

In [10]:
def genetic_algorithm(n, k, cas_izvajanja, t=4):
    '''funkcija v vsaki ponovitvi naredi novo populacijo z križanjem in mutiranjem'''
    populacija = zacetna_populacija(n, k)
    for i in range(cas_izvajanja):
        for graf in populacija:
            fitnes = fitness(graf)
            print(round(fitnes, 0))
            if fitnes < 0:
                print('Lema ne drži!')
                return graf
        populacija = nova_populacija(populacija,t)
    return 'Ne najdem protiprimera.'

In [20]:
genetic_algorithm(14, 13, 25, 4)

6
5
4
5
5
6
5
5
6
5


5
5
6
5


4
3
4
3
4
4
4
4
3
4
4
4
4
3


3
2
3
3
2
3
2
3
3
3
2
2
3
3


2
2
2
2
2
2
2
2
2
2
2
2
2
2


...

1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
